In [1]:
import mediapipe as mp
import cv2

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
file = 'images/manos.jpg'

In [4]:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  # Read an image from file
  image = cv2.imread(file)
  # Get image dimensions
  height, width, _ = image.shape
  # Flip the image horizontally for a later selfie-view display.
  image = cv2.flip(image, 1)
  # Convert the BGR image to RGB.
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # Process the image and get hand landmarks.
  results = hands.process(image_rgb)

  print('Handedness: ', results.multi_handedness)

  # Draw hand landmarks on the image.
  if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
      mp_drawing.draw_landmarks(
          image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

  # Convert the RGB image back to BGR.
  image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

Handedness:  [classification {
  index: 0
  score: 0.986842871
  label: "Left"
}
, classification {
  index: 1
  score: 0.95367527
  label: "Right"
}
]


I0000 00:00:1746631832.582849 2364516 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746631832.588260 2364801 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746631832.592276 2364801 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746631832.609901 2364802 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [5]:
def distance(p1, p2):
    return ((p2.x - p1.x) ** 2 + (p2.y - p1.y) ** 2) ** 0.5

In [6]:
def finger_raised(landmarks, finger_tip, finger_dip, finger_pip, finger_mcp, umbral=0.2):
    return ( 
            landmarks[finger_tip].y < landmarks[finger_dip].y and
            landmarks[finger_dip].y < landmarks[finger_pip].y and
            landmarks[finger_pip].y < landmarks[finger_mcp].y
    )

In [7]:
if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        # Draw hand landmarks on the image.
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        # Get the landmarks as list
        landmarks = hand_landmarks.landmark

        PULGAR_TIP = 4
        INDICE_TIP = 8
        MEDIO_TIP = 12
        ANULAR_TIP = 16
        MENIQUE_TIP = 20

        # Puntos para cada dedo (tip, dip, pip, mcp)
        fingers = {
            'pulgar': (PULGAR_TIP, 2, 1, 0),
            'indice': (INDICE_TIP, 6, 5, 0),
            'medio': (MEDIO_TIP, 10, 9, 0),
            'anular': (ANULAR_TIP, 14, 13, 0),
            'menique': (MENIQUE_TIP, 18, 17, 0)
        }

In [8]:
# Verify raised fingers
raised_fingers = []
for name, (tip, dip, pip, mcp) in fingers.items():
    if finger_raised(landmarks, tip, dip, pip, mcp):
        raised_fingers.append(name)

# Print the raised fingers
print("Fingers raised: ", raised_fingers)

Fingers raised:  ['pulgar', 'indice', 'medio', 'anular', 'menique']


In [9]:
# Show the image with landmarks
cv2.putText(image, "Fingers raised: " + ", ".join(raised_fingers), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

array([[[21, 22, 18],
        [21, 22, 18],
        [22, 23, 19],
        ...,
        [16, 17, 13],
        [16, 17, 13],
        [16, 17, 13]],

       [[21, 22, 18],
        [21, 22, 18],
        [22, 23, 19],
        ...,
        [16, 17, 13],
        [16, 17, 13],
        [16, 17, 13]],

       [[22, 23, 19],
        [22, 23, 19],
        [21, 22, 18],
        ...,
        [16, 17, 13],
        [16, 17, 13],
        [16, 17, 13]],

       ...,

       [[12, 13,  9],
        [12, 13,  9],
        [12, 13,  9],
        ...,
        [11, 12,  8],
        [11, 12,  8],
        [11, 12,  8]],

       [[12, 13,  9],
        [12, 13,  9],
        [12, 13,  9],
        ...,
        [11, 12,  8],
        [11, 12,  8],
        [11, 12,  8]],

       [[12, 13,  9],
        [12, 13,  9],
        [12, 13,  9],
        ...,
        [11, 12,  8],
        [11, 12,  8],
        [11, 12,  8]]], dtype=uint8)

In [ ]:
# Show the image with landmarks
cv2.imshow('Hand Landmarks', image)
cv2.waitKey(0)
cv2.destroyAllWindows()